In [219]:
import numpy as np
import pyaudio
import time
from pyldpc import make_ldpc, encode, decode, get_message

# Ses kayıt parametreleri
CHUNK = 1024*2
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

# Taşıyıcı frekansı
tasiyici_frekans = 1000




n = 32
d_v = 4
d_c = 8
snr = 10
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]
#>>> v = np.random.randint(2, size=k)

# PyAudio nesnesi oluştur
p = pyaudio.PyAudio()

# Çıkış akışını aç
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    output=True,
    frames_per_buffer=CHUNK
)

# Gönderilecek veri (binary)
v = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=np.int16)
y = encode(G, v, snr)
genisletilmis_veri = np.repeat(y, CHUNK // len(y))
#print(genisletilmis_veri[:16])

# Taşıyıcı dalga oluşturma
t = np.arange(0, len(genisletilmis_veri)) / RATE
tasiyici_dalga = np.sin(2 * np.pi * tasiyici_frekans * t)
m=tasiyici_dalga * (genisletilmis_veri * 2 - 1) 
# Frekans modülasyonu
modüle_edilmis_dalga = m*32767 #np.where(m>0,m * 32767,m*-32767)

# Modüle edilmiş dalgayı ses cihazına gönderme
for i in range(50):
	stream.write(modüle_edilmis_dalga.astype(np.int16).tobytes())
	time.sleep(0.0015)
# Akışı kapat
stream.stop_stream()
stream.close()
p.terminate()


In [198]:
>>> import numpy as np
>>> from pyldpc import make_ldpc, encode, decode, get_message
>>> n = 32
>>> d_v = 4
>>> d_c = 8
>>> snr = 1000
>>> H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
>>> k = G.shape[1]
>>> v = np.random.randint(2, size=k)
>>> y = encode(G, v, snr)
>>> d = decode(H, y, snr)
>>> x = get_message(G, d)
>>> assert abs(x - v).sum() == 0

In [199]:
print(len(y))

32


In [200]:
print(len(v))

19


In [197]:
k

1027

In [59]:
d = decode(H, y, 20)
x = get_message(G, d)
x


array([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0])

In [179]:
print(len(y))

2048


In [201]:
y

array([ 1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,
        1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
       -1.,  1., -1.,  1., -1., -1.])

In [208]:
m

array([-0.        , -4.02379232, -1.1710782 , ...,  4.19242495,
        1.43320713, -3.7753066 ])

In [206]:
aci = np.arcsin(0.5)  

In [207]:
modüle_edilmis_dalga

array([     -0.        , -131847.60304682,  -38372.7194361 , ...,
        137373.18829127,   46961.89798494, -123705.47121552])

In [209]:
m2=modüle_edilmis_dalga/32767

In [210]:
m2

array([-0.        , -4.02379232, -1.1710782 , ...,  4.19242495,
        1.43320713, -3.7753066 ])

In [211]:
t = np.arange(0, 2048) / RATE
tasiyici_dalga = np.sin(2 * np.pi * 10000 * t)




In [212]:
genis_veri=(m2/tasiyici_dalga +1)/2


/tmp/ipykernel_9541/141944224.py:1: RuntimeWarning: invalid value encountered in divide
  genis_veri=(m2/tasiyici_dalga +1)/2


array([ 0.        ,  0.98935543,  0.28794045, ..., -0.98087204,
       -0.33531735,  0.88328181])